In [2]:
import textract
import os
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import PunktSentenceTokenizer
from collections import Counter, defaultdict
from gensim import corpora, models, matutils
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
# Topic modeling

vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)



In [ ]:
vectorizer.vocabulary_

In [ ]:
vocab = {v: k for k, v in vectorizer.vocabulary_.iteritems()}
vocab

In [ ]:
vectorizer

In [ ]:
# remove words that appear only once
frequency = defaultdict(int)

for text in documents:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
          for text in documents]

# Create gensim dictionary object
dictionary = corpora.Dictionary(texts)

# Create corpus matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
lda = models.LdaModel(
    matutils.Sparse2Corpus(X, documents_columns=False),
    # or use the corpus object created with the dictionary in the previous frame!
    # corpus, 
    num_topics  =  3,
    passes      =  20,
    id2word     =  vocab
    # or use the gensim dictionary object!
    # id2word     =  dictionary
)

In [ ]:
lda.print_topics(num_topics=3, num_words=5)

In [ ]:
lda.get_document_topics(corpus[0])

In [ ]:
topics_labels = {
   0: "Family Stress",
   1: "Driving",
   2: "Food"
}

In [ ]:
doc_topics = [lda.get_document_topics(doc) for doc in corpus]

topic_data = []

for document_id, topics in enumerate(doc_topics):
    
    document_topics = []
    
    for topic, probability in topics:
       
        topic_data.append({
            'document_id':  document_id,
            'topic_id':     topic,
            'topic':        topics_labels[topic],
            'probability':  probability
        })

topics_df = pd.DataFrame(topic_data)
topics_df.pivot_table(values="probability", index=["document_id", "topic"]).T